In [4]:
import pandas as pd
import torch
import torchvision
from torchvision.io import read_image
from torchvision.utils import draw_bounding_boxes
from PIL import Image

In [5]:
#Change these file paths depending if your generating training or test labels
elephants_csv = 'datasets/AED_gsd/test_elephants_new.csv'
df_elephants = pd.read_csv(elephants_csv)
images = 'datasets/AED_gsd/images/test_images_bbox/'
output = 'datasets/AED_gsd/labels/test_images_bbox/'
print(df_elephants)

                                    image name     x     y
0     7d4d55723d3f6b8f174a88d36fc493d0b7fb6fc3  1719   592
1     04f53ca293e5037fc04f61e49b372d8f003c482d  1375  1226
2     04f53ca293e5037fc04f61e49b372d8f003c482d  1796   319
3     884fa549d6eaaf243d9db96bca0382f9068af146   946   777
4     884fa549d6eaaf243d9db96bca0382f9068af146   952   799
...                                        ...   ...   ...
2965  c8f33a304bd04467f40271ca2cf1bf011128f196   710   901
2966  69fab1105c5709822edd34975aa0faaa5d922866   901  1174
2967  69fab1105c5709822edd34975aa0faaa5d922866   845  1420
2968  69fab1105c5709822edd34975aa0faaa5d922866   871  1119
2969  69fab1105c5709822edd34975aa0faaa5d922866   908  1074

[2970 rows x 3 columns]


In [19]:
#Loop through each image
for image_name in df_elephants['image name'].unique():
    #Accumulate rows for each image
    data = []
    
    #Get the a dataframe that has all rows with this image
    df_image = df_elephants[df_elephants['image name'] == image_name]

    #Load image (so we can get image dimensions)
    image_path = f'{images}{image_name}.jpg'
    img = Image.open(image_path) #use PIL to open image
    img_width, img_height = img.size

    #Loop through each elephant in current image
    for index, row in df_image.iterrows():
        #Calculate normalized bbox coordinates for yolo
        
        #THIS WILL NEED TO BE CHANGED IF USING CANNY DETECTION
        #Rather than using the elephants csv file that is image, x, y. You would use the canny detection csv file and convert
        #the bbox coordinates given there to the format yolo wants
        
        x = row['x']
        y = row['y']
        width = 25
        height = 25

        x_center = x/img_width
        y_center = y/img_height
        norm_width = width/img_width
        norm_height = height/img_height

        if(x_center > 1):
            x_center = 1
        if(y_center > 1):
            y_center = 1

        data.append({'class': 0, 'x_center': x_center, 'y_center': y_center, 'width': norm_width, 'height': norm_height})
        
    #Output to txt file
    df_data = pd.DataFrame(data)
    #print(df_data)
    
    filename = output + image_name + '.txt'
    df_data.to_csv(filename, sep='\t', index=False, header=None)

Instructions for running yolov5 on our dataset:

1. Download yolov5 by running these commands:

git clone https://github.com/ultralytics/yolov5
   
cd yolov5
   
pip install -r requirements.txt

This creates a directory called yolov5.

2. In the same directory that you installed the yolov5 directory, create a new directory called datasets.
  
   In this directory, create another directory called AED_gsd.

   In the AED_gsd directory, create two new directories called images and labels.

   Put the 2 folders containing the training_images and test_images (the ones with consistent GSD) into the images directory

   Use the code in this notebook to generate labels for the both the training_images and test_images. The folders within labels that contain the labels need to be named the same thing as the folders that contain the images. Just put this notebook in the same directory where the yolov5 and datasets directories are contained.

3. Put the provided AED.yaml and AED_test.yaml files into the data folder within the yolov5 directory.

4. Run these commands:

python train.py --img 640 --epochs 300 --batch-size 8 --data AED.yaml --weights yolov5n.pt

python val.py --img 640 --weights runs/train/exp10/weights/best.pt --data AED_test.yaml

python detect.py --source ../datasets/AED_gsd/images/test_images/ --weights runs/train/exp10/weights/best.pt --img 640

The first command trains the model. Depending on your PC, you can up the batch size, up the img size (1280), and use the bigger pretrained models (weights). The results are saved in yolov5/runs/train/exp# (exp# is just the most recent run). This also takes like 15 hours probably.

The second command will test the model and see how well it performs. The results are saved in yolov5/runs/val/exp#.

The third command will actually try to detect the elephants in the test images using the trained model. This outputs all the images into yolov5/runs/detect/exp# with the detected bounding boxes. You can change what it outputs and saves using command line args that you can find in detect.py.